# 1. 讀取檔案

In [97]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# 使用 pandas 的 read_csv 函式讀取訓練資料
df = pd.read_csv("data/train.csv")

In [98]:
df.head(10)


,id,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
1,18,27.0,4,97.0,88.0,2130,14.5,70,japan,datsun pl510
2,376,37.0,4,91.0,68.0,2025,18.2,82,japan,mazda glc custom l
3,248,36.1,4,91.0,60.0,1800,16.4,78,japan,honda civic cvcc
4,177,23.0,4,115.0,95.0,2694,15.0,75,europe,audi 100ls
5,63,14.0,8,400.0,175.0,4385,12.0,72,usa,pontiac catalina
6,246,32.8,4,78.0,52.0,1985,19.4,78,japan,mazda glc deluxe
7,352,29.9,4,98.0,65.0,2380,20.7,81,usa,ford escort 2h
8,108,20.0,4,97.0,88.0,2279,19.0,73,japan,toyota carina
9,90,12.0,8,429.0,198.0,4952,11.5,73,usa,mercury marquis brougham


In [99]:
def FindCom(df):
    comp=[i.split(" ")[0] for i in df['name']]
    return comp

def Firm(df):
    comp=FindCom(df)
    # print(set(comp))
    df['name']=comp
    df['name']=df['name'].fillna(df['name'].mode())
    data_dum = pd.get_dummies(df['name'])
    df=pd.concat([df,data_dum],axis=1)
    df['name']=df['name'].fillna(df['name'].mode())
    return df
df_clean=Firm(df)

In [100]:

df_clean

,id,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,...,pontiac,renault,saab,subaru,toyota,triumph,vokswagen,volkswagen,volvo,vw
0,3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc,...,False,False,False,False,False,False,False,False,False,False
1,18,27.0,4,97.0,88.0,2130,14.5,70,japan,datsun,...,False,False,False,False,False,False,False,False,False,False
2,376,37.0,4,91.0,68.0,2025,18.2,82,japan,mazda,...,False,False,False,False,False,False,False,False,False,False
3,248,36.1,4,91.0,60.0,1800,16.4,78,japan,honda,...,False,False,False,False,False,False,False,False,False,False
4,177,23.0,4,115.0,95.0,2694,15.0,75,europe,audi,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,71,19.0,3,70.0,97.0,2330,13.5,72,japan,mazda,...,False,False,False,False,False,False,False,False,False,False
314,106,12.0,8,350.0,180.0,4499,12.5,73,usa,oldsmobile,...,False,False,False,False,False,False,False,False,False,False
315,270,21.1,4,134.0,95.0,2515,14.8,78,japan,toyota,...,False,False,False,False,True,False,False,False,False,False
316,348,37.7,4,89.0,62.0,2050,17.3,81,japan,toyota,...,False,False,False,False,True,False,False,False,False,False


In [101]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            318 non-null    int64  
 1   mpg           318 non-null    float64
 2   cylinders     318 non-null    int64  
 3   displacement  318 non-null    float64
 4   horsepower    313 non-null    float64
 5   weight        318 non-null    int64  
 6   acceleration  318 non-null    float64
 7   model_year    318 non-null    int64  
 8   origin        318 non-null    object 
 9   name          318 non-null    object 
dtypes: float64(4), int64(4), object(2)
memory usage: 25.0+ KB
None


In [102]:
m=df.iloc[:,1:7].corr()
print(m)

                   mpg  cylinders  displacement  horsepower    weight  \
mpg           1.000000  -0.770459     -0.801543   -0.772308 -0.827333   
cylinders    -0.770459   1.000000      0.951825    0.839187  0.893808   
displacement -0.801543   0.951825      1.000000    0.888972  0.929479   
horsepower   -0.772308   0.839187      0.888972    1.000000  0.855325   
weight       -0.827333   0.893808      0.929479    0.855325  1.000000   
acceleration  0.390101  -0.479743     -0.512604   -0.672726 -0.380938   

              acceleration  
mpg               0.390101  
cylinders        -0.479743  
displacement     -0.512604  
horsepower       -0.672726  
weight           -0.380938  
acceleration      1.000000  


# 2.資料預處理

In [103]:
# 直接刪除任何含有缺失值的整行資料
# HINT：有更好的預處理填補方式嗎？
#刪掉完全重複
df_clean=df_clean.drop_duplicates()
#檢查空值
# print(df_clean.isnull().sum().sum())


#嘗試填入空值
"""df.info()"""
"""for i in range(3,8):
    df_clean.iloc[:,i]=df_clean.iloc[:,i].fillna(df.iloc[:,i].median())"""
#填入數字
"""df_clean['cylinders']=df_clean['cylinders'].fillna(df_clean['cylinders'].mode())
df_clean['origin']=df_clean['origin'].fillna(df_clean['origin'].mode())"""
# df_clean.iloc[:,3:7]=df_clean.iloc[:,3:7].fillna(df_clean.iloc[:,3:7].median())
df_clean.iloc[:,4]=df_clean.iloc[:,4].fillna(df_clean.iloc[:,4].median())
# df_clean['name']=df_clean['name'].fillna(df_clean['name'].mode())
#去除極端值

def STD(df_clean,i):
    u=-1
    while u!=len(df_clean[i]):
        u=len(df_clean[i])
        print(len(df_clean))
        desc=df_clean[i].describe()
        std=desc['std']
        mean=desc['mean']
        up=mean+3*std
        lower=mean-3*std
        Outer_df=df_clean[(df_clean[i]>up) | (df_clean[i]<lower)]
        df_clean=df_clean.drop(index=Outer_df.index)
    return df_clean
df_clean=STD(df_clean,'acceleration')


    
    #df_clean[i].iloc[Outer_df.index]=df_clean[i].mode()


#NULL檢查
"""print(df_clean.isnull())
missing_count = df_clean.isnull().sum().sum()
print(missing_count)"""

#標準化
# from sklearn.preprocessing import StandardScaler
# def ST(df):
#     scaler = StandardScaler()
#     df.loc[:,'cylinders':'model_year']=scaler.fit_transform(df.loc[:,'cylinders':'model_year'])
#     return df
# df_clean=ST(df_clean)
# print(df_clean['cylinders'].describe()['std'])



318
317


'print(df_clean.isnull())\nmissing_count = df_clean.isnull().sum().sum()\nprint(missing_count)'

In [104]:
data_dum = pd.get_dummies(df_clean['origin'])
data_dum=data_dum.drop('europe',axis=1)
df_clean=pd.concat([df_clean,data_dum],axis=1)
# print(df_clean)


# 3.特徵工程

In [105]:
# 定義要用來預測的特徵欄位
# HINT：這串列遺漏了部分特徵，另外，有方法額外加上新的特徵嗎？
#features = ['weight', 'acceleration', 'model_year', 'cylinders', 'displacement', 'horsepower']
features = ['weight', 'model_year', 'displacement','japan','usa','acceleration']
i=(df_clean.columns.to_list().index('name'))
comp=df_clean.iloc[:,i+1:].columns.to_list()
features+=comp
# 定義我們要預測的目標欄位
target = 'mpg'

# 從乾淨的資料中選取 X 和 y
X = df_clean[features]
y = df_clean[target]


# 4. 訓練模型

In [106]:
# 分割訓練集與測試集 (用於本地評估模型好壞)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立並訓練模型
model = LinearRegression()
model.fit(X_train, y_train)

# 進行預測與評估
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

print("模型已訓練完成！")
print(f"訓練誤差 (Train RMSE): {train_rmse:.4f} MPG")
print(f"測試誤差 (Test RMSE):  {test_rmse:.4f} MPG")
print("\n--- 模型學到的關係 ---")

# 各個特徵的權重
for feature, coef in zip(features, model.coef_):
    print(f"特徵 '{feature}' 的權重: {coef:.4f}")

模型已訓練完成！
訓練誤差 (Train RMSE): 2.9289 MPG
測試誤差 (Test RMSE):  4.0288 MPG

--- 模型學到的關係 ---
特徵 'weight' 的權重: -0.0063
特徵 'model_year' 的權重: 0.7397
特徵 'displacement' 的權重: 0.0023
特徵 'japan' 的權重: -0.7623
特徵 'usa' 的權重: -1.0529
特徵 'acceleration' 的權重: -0.0352
特徵 'amc' 的權重: -1.4857
特徵 'audi' 的權重: -3.6479
特徵 'bmw' 的權重: 0.8619
特徵 'buick' 的權重: -0.2965
特徵 'cadillac' 的權重: 2.0586
特徵 'capri' 的權重: -0.2918
特徵 'chevroelt' 的權重: -0.2984
特徵 'chevrolet' 的權重: 0.0891
特徵 'chevy' 的權重: -0.3043
特徵 'chrysler' 的權重: -3.2405
特徵 'datsun' 的權重: 2.1901
特徵 'dodge' 的權重: -0.5259
特徵 'fiat' 的權重: 1.0280
特徵 'ford' 的權重: -0.9174
特徵 'hi' 的權重: 1.5578
特徵 'honda' 的權重: 2.8420
特徵 'maxda' 的權重: -8.3739
特徵 'mazda' 的權重: 2.2120
特徵 'mercedes' 的權重: 1.9280
特徵 'mercedes-benz' 的權重: 0.5806
特徵 'mercury' 的權重: -1.6698
特徵 'oldsmobile' 的權重: 1.1077
特徵 'opel' 的權重: -2.0720
特徵 'peugeot' 的權重: -0.7557
特徵 'plymouth' 的權重: 0.7931
特徵 'pontiac' 的權重: 2.3711
特徵 'renault' 的權重: 2.6966
特徵 'saab' 的權重: -3.2007
特徵 'subaru' 的權重: -0.2255
特徵 'toyota' 的權重: 0.5930
特徵 'triumph' 的權重:

In [107]:
print(df_clean)

      id   mpg  cylinders  displacement  horsepower  weight  acceleration  \
0      3  16.0          8         304.0       150.0    3433          12.0   
1     18  27.0          4          97.0        88.0    2130          14.5   
2    376  37.0          4          91.0        68.0    2025          18.2   
3    248  36.1          4          91.0        60.0    1800          16.4   
4    177  23.0          4         115.0        95.0    2694          15.0   
..   ...   ...        ...           ...         ...     ...           ...   
313   71  19.0          3          70.0        97.0    2330          13.5   
314  106  12.0          8         350.0       180.0    4499          12.5   
315  270  21.1          4         134.0        95.0    2515          14.8   
316  348  37.7          4          89.0        62.0    2050          17.3   
317  102  26.0          4          97.0        46.0    1950          21.0   

     model_year  origin        name  ...   saab  subaru  toyota  triumph  \

# 5.輸出提交檔案

In [ ]:
# 讀取需要進行預測的測試檔案 test.csv
df_test = pd.read_csv("data/test.csv")

#NEW
data_dum = pd.get_dummies(df_test['origin'])
data_dum=data_dum.drop('europe',axis=1)
df_test=pd.concat([df_test,data_dum],axis=1)

# 對測試資料進行預處理
# HINT：如果前面使用了其他的預處理方式，這邊要如何修改？
df_test=Firm(df_test)
# for i in (df_clean.columns):
#     if i not in df_test.columns:
#         df_test=pd.concat([df_test,pd.DataFrame({i:[False]*len(df_test)})],axis=1)

# print(df_clean.columns)
# print(df_test.columns)
for i in (df_clean.columns):
    if i not in df_test.columns:
        df_test=pd.concat([df_test,pd.DataFrame({i:[False]*len(df_test)})],axis=1)

# print(df_test.tail(20))
#
df_test['cylinders']=df_test['cylinders'].fillna(df_test['cylinders'].mode())
df_test['origin']=df_test['origin'].fillna(df_test['origin'].mode())
df_test.iloc[:,3:7]=df_test.iloc[:,3:7].fillna(df_test.iloc[:,3:7].median())

#




#NEW2
# data_dum = pd.get_dummies(df_test['name'])
# df_test=pd.concat([df_test,data_dum],axis=1)
# 使用訓練好的模型，對測試資料進行預測
predictions = model.predict(df_test[features])

# 建立一個新的 DataFrame
submission_df = pd.DataFrame({'Id':df_test['id'], 'mpg': predictions})

# 保存為 submission.csv
submission_df.to_csv('submission.csv', index=False)
print("提交文件 'submission.csv' 已成功生成！")

Index(['id', 'mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model_year', 'origin', 'name', 'amc', 'audi', 'bmw',
       'buick', 'cadillac', 'capri', 'chevroelt', 'chevrolet', 'chevy',
       'chrysler', 'datsun', 'dodge', 'fiat', 'ford', 'hi', 'honda', 'maxda',
       'mazda', 'mercedes', 'mercedes-benz', 'mercury', 'oldsmobile', 'opel',
       'peugeot', 'plymouth', 'pontiac', 'renault', 'saab', 'subaru', 'toyota',
       'triumph', 'vokswagen', 'volkswagen', 'volvo', 'vw', 'japan', 'usa'],
      dtype='object')
Index(['id', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model_year', 'origin', 'name', 'japan', 'usa', 'amc',
       'audi', 'buick', 'cadillac', 'chevrolet', 'chrysler', 'datsun', 'dodge',
       'fiat', 'ford', 'honda', 'maxda', 'mazda', 'mercury', 'nissan',
       'oldsmobile', 'peugeot', 'plymouth', 'pontiac', 'saab', 'toyota',
       'toyouta', 'volkswagen', 'volvo', 'vw'],
      dtype='object')
Index

# 6. 報告

姓名：__________ 學號：__________

第一部分：準確度分數 (Accuracy Scores) (1分)  
我的準確度分數：__________  

第二部分：我的實驗記錄 (My Experiment Log) (3分)  
請記錄你做了哪些嘗試來提升分數，至少記錄兩次不同的嘗試。  
【實驗 1】  
    我做的修改：__________________________________________________________________________________  
    結果與觀察 (分數變化、心得等)：__________________________________________________________________  
    該次實驗分數： ____________  
【實驗 2】  
    我做的修改：__________________________________________________________________________________  
    結果與觀察 (分數變化、心得等)：__________________________________________________________________  
    該次實驗分數： ____________  

第三部分：總結與心得 (Conclusion & Reflection) (2分)  
請撰寫一段約 50-100 字的心得總結。內容需包含：  
(1) 你認為本次實驗中，提升準確率最有效的修改是什麼。  
(2) 這次不斷嘗試與修正的過程，帶給你最大的學習與啟發。  
內容：______________________________________________    